In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
from pymongo import MongoClient

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser1"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)


## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'  # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Center(html.B(html.H1('Grazioso Salvare'))),
    html.Center(
        html.Img(src=f'data:image/png;base64,{encoded_image.decode()}', style={'width': '400px', 'height': 'auto'}),
    ),
    html.Center(html.H4("Brett Nottmeier")),
    html.Hr(),

    # FIX ME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    html.Div([
        dcc.RadioItems(
            id='filter-options',  # Changed to a valid ID
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mount'},
                {'label': 'Disaster/Individual Tracking', 'value': 'disaster'},
                {'label': 'All Rescues', 'value': 'All'},
                
            ],
            value='All',
            labelStyle={'display': 'inline-block'}
        )
    ]),

    html.Hr(),

    #FIX ME: Set up the features for your interactive data table to make it user-friendly for your client
    # If you completed the Module Six Assignment, you can copy in the code you created here
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        page_action="native",
        page_current=0,
        page_size=10,
    ),

    html.Br(),
    html.Hr(),

    html.Div(id='map-id', className='col s12 m6'),

    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')  # Ensure only one `map-id` exists
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-id', 'data'),
              [Input('filter-options', 'value')])  
def update_dashboard(filter_type):
    ## FIX ME Add code to filter interactive data table with MongoDB queries
    query = {}
    
    if filter_type == 'water':
        f_filtered = pd.DataFrame.from_records(db.read(query))
        query = {"animal_type": "Dog", 
                 "breed":{"$in" :["Labrador Retriever Mix", "Chesapeake Bay Retriever", "NewFoundLand"]},
                 "sex_upon_outcome": "Intact Female",
                 "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}}
        
    elif filter_type == 'mount':
        query = {"animal_type": "Dog",
                 "breed":{"$in" :["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
                 "sex_upon_outcome": "Intact Male",
                 "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}}
        
    elif filter_type == 'disaster':
        query = {"animal_type": "Dog", 
                 "breed":{"$in" :["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
                 "sex_upon_outcome": "Intact Male",
                 "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}}
    else:
        query = {}  # Return all rescues
    
    df_filtered = pd.DataFrame.from_records(db.read(query))
    
    if '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'], inplace=True)  # Drop '_id' column
    
    return df_filtered.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # FIX ME Add code for chart of your choice (e.g. pie chart)
    if viewData is None:
        return html.Div("No data to display")

    dff = pd.DataFrame(viewData)
    if 'breed' in dff.columns:
        return [
            dcc.Graph(
                figure=px.pie(dff, names='breed', title='Preferred Dog Breeds')
            )
        ]
    else:
        return html.Div("No breed data available")
    
# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  

    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    row = index[0]
    
    if row >= len(dff):
        return html.Div("Invalid row selected")
    
    # Check if latitude and longitude columns exist
    if 'location_lat' not in dff.columns or 'location_long' not in dff.columns:
        return html.Div("No location data available")
    
    lat = dff.iloc[row]['location_lat']
    lon = dff.iloc[row]['location_long']
    
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tooltip and popup
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(dff.iloc[row]['breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row]['name'])
                ])
            ])
        ])
    ]

# Run the app
app.run_server(debug=True)

Dash app running on http://127.0.0.1:13108/
